In [ ]:
import nlpaug.augmenter.word as naw
from textaugment import Translate
from textaugment import Wordnet
from textaugment import EDA
import nlpaug.flow as naf
import pandas as pd
import numpy as np
import random
import csv

### BERT

In [ ]:
aug_bert = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased',
    action='substitute',
    top_k=20
    )

aug_bert.augment('In the afternoon, John is going to town')

### Word2Vec

In [ ]:
aug_w2v = naw.WordEmbsAug(
    model_type='glove',
    model_path='./model/glove.6B.300d.txt',
    action="substitute")

aug_w2v.augment('In the afternoon, John is going to town')

### RTT-based augmentation

In [ ]:
translate = Translate(src="en", to="ru")
translate.augment('In the afternoon, John is going to town')

### WordNet-based augmentation

In [ ]:
wordnet = Wordnet(v=False ,n=True, p=0.5)
wordnet.augment('In the afternoon, John is going to town')

### Easy data augmentation

In [ ]:
eda=EDA()
print(eda.synonym_replacement("yesterday I went to the university and after that I went to the training"))
print(eda.random_swap("yesterday I went to the university and after that I went to the training"))
print(eda.random_insertion("yesterday I went to the university and after that I went to the training"))

### Generation

In [ ]:
functions = [
    aug_bert.augment,
    aug_w2v.augment,
    wordnet.augment,
    eda.synonym_replacement,
    eda.random_swap,
    eda.random_insertion
]

def generate_text(functions, text):
    return random.choice(functions)(text)

generate_text(functions, 'Airlines/Aviation')

In [ ]:
df = pd.read_csv("data/train.csv")
df = df.drop(["Unnamed: 0"],axis=1)

df = df[df['fraudulent'] > 0]

In [ ]:
for col in ['employment_type', 'required_experience', 'required_education', 'function']:
    print(df.groupby(col)[col].count())
    print(sum(df.groupby(col)[col].count()))
    print(len(df.groupby(col)[col]))
    print()

In [ ]:
dic = dict()
employment_type = dict()
grp = df.groupby('employment_type')['employment_type'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    employment_type[previous] = dic[key]

In [ ]:
dic = dict()
required_experience = dict()
grp = df.groupby('required_experience')['required_experience'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    required_experience[previous] = dic[key]

In [ ]:
dic = dict()
function = dict()
grp = df.groupby('function')['function'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    function[previous] = dic[key]

In [ ]:
dic = dict()
required_education = dict()
grp = df.groupby('required_education')['required_education'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    required_education[previous] = dic[key]

In [ ]:
def get_random_dict_value(dic):
    rndm = random.random()
    for key in dic:
        if rndm < key:
            return random.choice(dic[key])

In [ ]:
columns = ['title', 'location', 'department', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'salary_range', 'telecommuting', 'has_company_logo', 'has_questions']
statistic_columns = ['employment_type', 'required_experience', 'required_education', 'function']
statistics = {'employment_type': 0.28060522696011003, 'required_experience': 0.49105914718019256, 'required_education': 0.515818431911967,'industry': 0.3177441540577717,'function': 0.38514442916093533}

data = dict()
repetitions = 20

for column in columns:
    data[column] = list(df[column].values)

text = 'title\tlocation\tdepartment\tdescription\trequirements\tbenefits\temployment_type\trequired_experience\trequired_education\tindustry\tfunction\tsalary_range\ttelecommuting\thas_company_logo\thas_questions\n'
for index in range(len(df['title'])):
    print(index)
    for rep in range(repetitions):
        for col in columns:
            if col in ['salary_range', 'telecommuting', 'has_company_logo', 'has_questions']:
                try:
                    text += str(df[col][index])
                except:
                    text += " \t"
            elif col == 'employment_type':
                if random.random() > 0.28060522696011003:
                    text += get_random_dict_value(employment_type) + '\t'
                else:
                    text += " \t"
            elif col == 'required_experience':
                if random.random() > 0.49105914718019256:
                    text += get_random_dict_value(required_experience) + '\t'
                else:
                    text += " \t"
            elif col == 'required_education':
                if random.random() > 0.515818431911967:
                    text += get_random_dict_value(required_education) + '\t'
                else:
                    text += " \t"
            elif col == 'function':
                if random.random() > 0.38514442916093533:
                    text += get_random_dict_value(function) + '\t'
                else:
                    text += " \t"
            else:
                try:
                    text += generate_text(functions, data[col][index]) + '\t'
                except:
                    text += " \t"
        text = text[:-1] + '\n'

with open('augmented+all.csv', 'w') as f:
    f.write(text)

In [ ]:
pd.read_csv("augmented+all.csv", delimiter='\t')

### BERT

In [2]:
aug_bert = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased',
    action='substitute',
    top_k=20
    )

aug_bert.augment('In the afternoon, John is going to town')

'across the afternoon, charlie is driving to town'

### Word2Vec

In [3]:
aug_w2v = naw.WordEmbsAug(
    model_type='glove',
    model_path='./model/glove.6B.300d.txt',
    action="substitute")

aug_w2v.augment('In the afternoon, John is going to town')

'In the afternoon, John appears them to hometown'

### RTT-based augmentation

In [4]:
translate = Translate(src="en", to="ru")
translate.augment('In the afternoon, John is going to town')

'john goes to the city during the day'

### WordNet-based augmentation

In [5]:
wordnet = Wordnet(v=False ,n=True, p=0.5)
wordnet.augment('In the afternoon, John is going to town')

'in the afternoon, lavatory is going to town'

### Easy data augmentation

In [6]:
eda=EDA()
print(eda.synonym_replacement("yesterday I went to the university and after that I went to the training"))
print(eda.random_swap("yesterday I went to the university and after that I went to the training"))
print(eda.random_insertion("yesterday I went to the university and after that I went to the training"))

yesterday ace went to the university and after that ace went to the training
yesterday I went to the university and the that I went to after training
yesterday I went to the university fit and after that I went to the training


### Generation

In [7]:
functions = [
    aug_bert.augment,
    aug_w2v.augment,
    wordnet.augment,
    eda.synonym_replacement,
    eda.random_swap,
    eda.random_insertion
]

def generate_text(functions, text):
    return random.choice(functions)(text)

generate_text(functions, 'Airlines/Aviation')

'Airlines/Aviation'

In [8]:
df = pd.read_csv("data/train.csv")
df = df.drop(["Unnamed: 0"],axis=1)

df = df[df['fraudulent'] > 0]

In [9]:
for col in ['employment_type', 'required_experience', 'required_education', 'function']:
    print(df.groupby(col)[col].count())
    print(sum(df.groupby(col)[col].count()))
    print(len(df.groupby(col)[col]))
    print()

employment_type
Contract      38
Full-time    412
Other         15
Part-time     57
Temporary      1
Name: employment_type, dtype: int64
523
5

required_experience
Associate            37
Director             15
Entry level         154
Executive             8
Internship            9
Mid-Senior level     96
Not Applicable       51
Name: required_experience, dtype: int64
370
7

required_education
Associate Degree                       6
Bachelor's Degree                     83
Certification                         16
Doctorate                              1
High School or equivalent            144
Master's Degree                       26
Professional                           3
Some College Coursework Completed      2
Some High School Coursework           17
Unspecified                           54
Name: required_education, dtype: int64
352
10

function
Accounting/Auditing       24
Administrative            98
Advertising                5
Art/Creative               1
Business Development

In [10]:
dic = dict()
employment_type = dict()
grp = df.groupby('employment_type')['employment_type'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    employment_type[previous] = dic[key]

In [11]:
dic = dict()
required_experience = dict()
grp = df.groupby('required_experience')['required_experience'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    required_experience[previous] = dic[key]

In [12]:
dic = dict()
function = dict()
grp = df.groupby('function')['function'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    function[previous] = dic[key]

In [13]:
dic = dict()
required_education = dict()
grp = df.groupby('required_education')['required_education'].count()

for i in range(len(grp)):
    key = grp.values[i] / sum(grp)
    value = list(grp.keys())[i]
    
    if key not in dic:
        dic[key] = [value]
    else:
        dic[key].append(value)
dic = dict(sorted(dic.items()))

previous = 0
for key in dic.keys():
    previous += key * len(dic[key])
    required_education[previous] = dic[key]

In [14]:
def get_random_dict_value(dic):
    rndm = random.random()
    for key in dic:
        if rndm < key:
            return random.choice(dic[key])

In [40]:
columns = ['title', 'location', 'department', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'salary_range', 'telecommuting', 'has_company_logo', 'has_questions']
statistic_columns = ['employment_type', 'required_experience', 'required_education', 'function']
statistics = {'employment_type': 0.28060522696011003, 'required_experience': 0.49105914718019256, 'required_education': 0.515818431911967,'industry': 0.3177441540577717,'function': 0.38514442916093533}

data = dict()
repetitions = 20

for column in columns:
    data[column] = list(df[column].values)

text = 'title\tlocation\tdepartment\tdescription\trequirements\tbenefits\temployment_type\trequired_experience\trequired_education\tindustry\tfunction\tsalary_range\ttelecommuting\thas_company_logo\thas_questions\n'
for index in range(len(df['title'])):
    print(index)
    for rep in range(repetitions):
        for col in columns:
            if col in ['salary_range', 'telecommuting', 'has_company_logo', 'has_questions']:
                try:
                    text += str(df[col][index])
                except:
                    text += " \t"
            elif col == 'employment_type':
                if random.random() > 0.28060522696011003:
                    text += get_random_dict_value(employment_type) + '\t'
                else:
                    text += " \t"
            elif col == 'required_experience':
                if random.random() > 0.49105914718019256:
                    text += get_random_dict_value(required_experience) + '\t'
                else:
                    text += " \t"
            elif col == 'required_education':
                if random.random() > 0.515818431911967:
                    text += get_random_dict_value(required_education) + '\t'
                else:
                    text += " \t"
            elif col == 'function':
                if random.random() > 0.38514442916093533:
                    text += get_random_dict_value(function) + '\t'
                else:
                    text += " \t"
            else:
                try:
                    text += generate_text(functions, data[col][index]) + '\t'
                except:
                    text += " \t"
        text = text[:-1] + '\n'

with open('augmented+all.csv', 'w') as f:
    f.write(text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [41]:
pd.read_csv("augmented+all.csv", delimiter='\t')

,title,location,department,description,requirements,benefits,employment_type,required_experience,required_education,industry,function,salary_range,telecommuting,has_company_logo,has_questions
0,Client Audit coordinator Coordinator,"us, tx, austin",whole COMPLIANCE UNIT 701,Description SUMMARY: The Client Audit Coordina...,High school diploma or equivalent; Bachelor's ...,we offer a competitive and comprehensive ambit...,Full-time,,Doctorate,financial intelligence,Administrative,,,,
1,national audit coordinator,"us, ny, ny",compliance COMPLIANCE UNIT 701,Description SUMMARY: The Client Audit Coordina...,high school diploma or equivalent; bachelor's ...,nurses offer a competitive relatively comprehe...,Full-time,,Bachelor's Degree,fiscal Services,Engineering,,,,
2,node audit coordinator,"Austin TX, US,",701 UNIT COMPLIANCE,Description SUMMARY: The Client Audit Coordina...,High school diploma or equivalent; Bachelor's ...,we offer a competitive and comprehensive compa...,,,Bachelor's Degree,financial assistance,Other,,,,
3,client Audit Coordinator,"US, TX, capital of texas",compliance package 10,Description SUMMARY: The Client Audit Coordina...,High school diploma or equivalent; Bachelor's ...,we offer a competitive and comprehensive range...,,,High School or equivalent,fiscal Financial Services,Engineering,,,,
4,Client scrutinize Coordinator,"US, TX, Austin",COMPLIANCE UNIT compliance 701,Description SUMMARY: The Client Audit Coordina...,High school diploma or equivalent; Bachelor's ...,We offer in competitive and evaluation typical...,,Entry level,,financial services,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14525,Administrative administrative Assistant,"us, fl, miami",,Administrative Assistant provides personal adm...,Education and Experience: Usually every ago 3 ...,"health, dental, life and ad&amp;d insurance, e...",Other,,Unspecified,Accounting,Engineering,,,,
14526,Assistant Administrative,"US, FL, Miami",,Administrative Assistant provides personal adm...,Education and Experience:Usually at least 3 ye...,"Health, Dental, Life and AD&amp;D employee Ins...",Full-time,,Bachelor's Degree,pdf,,,,,
14527,administrative units,"us, fl, miami",,Administrative Assistant provides personal adm...,education support experience : usually at leas...,"health, dental, life and ad&amp;d insurance, e...",Full-time,Entry level,Associate Degree,°,Sales,,,,
14528,Administrative adjunct,"Miami FL, US,",,Administrative Assistant provides personal adm...,Education and Experience: Usually closing leas...,"health, dental, life and ad&amp;d insurance, e...",Contract,,Associate Degree,answer for,,,,,
